In [1]:
import pandas as pd
import numpy as np
import string
import csv

**steps**
    - need index to extract full text -> `review_text['text'][1]`
    - after extraction -> filtering/cleaning steps
    - filtered data should compose a new dataframe
        - columns: `text`, `business_id`

In [2]:
def clean_text(text):
    # remove punctuations
    clean_text = [char for char in text if char not in string.punctuation]
    clean_text = ''.join(clean_text)
    # lower case all text
    clean_text = clean_text.lower()
    
    return clean_text

In [3]:
chunk_size = 100000
raw_data_path = 'yelp_academic_dataset_review.csv'
clean_data_path = 'yelp_review_cleaned.csv'
column_names = ['business_id', 'text']

def read_csv(ds_name, chunk_size):
    
    for chunk in pd.read_csv(ds_name, chunksize=chunk_size):
        return chunk

review_text = read_csv(raw_data_path, chunk_size)

with open(clean_data_path, 'w') as csv_file:
    csv_file = csv.writer(csv_file)
    csv_file.writerow(column_names)
    
    for idx in range(chunk_size):
        cleaned_text = clean_text(review_text['text'][idx])
        business_id = review_text['business_id'][idx]
        # write into new csv file
        csv_file.writerow([business_id, cleaned_text])

In [4]:
clean_text = pd.read_csv(clean_data_path)

In [5]:
clean_text.tail()

,business_id,text
99995,b'3cLBtLFiH8IRdlEy9S8RiQ',bi figured i would try this place based on so ...
99996,b'1V5AWwFsxrlUsSo2B6MEEA',bwe were there for lunch on april 26th we opte...
99997,b'w4vP6vhfb65Te66zOO2Hug',bstopped in here after returning from a trip a...
99998,b'8u_pdcdN66D1dmIZrNK55w',bwho doesnt love wendys this one however is in...
99999,b'5Qt0Zo4z6pGLFfscU5gS3w',bunfortunately this is one of the worst of the...
